# Extract audio features from each song

In [3]:
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys

from tqdm.notebook import tqdm
import pandas as pd
import os
from time import sleep

#libraries for language extraction
#from spacy_langdetect import LanguageDetector
#import spacy
#from textblob import TextBlob
#import langid
#import cld3
#from langdetect import detect

In [4]:
SPOTIPY_CLIENT_ID='a4fdae2cec0c4755bd305bc64a0aa7ea'
SPOTIPY_CLIENT_SECRET='480fac44cf5c40ef854ce3725daf7b7a'

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

In [5]:
songs_ts= pd.read_csv(os.path.join('data', 'songs_timeseries_2017_2021.csv'), index_col=0)
songs_ts.head()

,song_id,num_day,total_countries,current_countries,inc,dec
0,4O4Z8VFczL8MxIOmqVWc1b,1,1,1,1,0
1,4O4Z8VFczL8MxIOmqVWc1b,2,2,1,1,1
2,4O4Z8VFczL8MxIOmqVWc1b,3,2,1,0,1
3,4O4Z8VFczL8MxIOmqVWc1b,4,2,1,0,1
4,4O4Z8VFczL8MxIOmqVWc1b,5,2,1,0,1


In [6]:
ranking_df = pd.read_csv(os.path.join('data','ranking_all_2017_2021.csv'), parse_dates=['Date', 'release_date'], index_col=0)
ranking_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,Ecuador,3AEZUABDXNtecAOSC1qTfo,2017-01-01
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,Ecuador,6mICuAdrwEjh6Y6lroV2Kg,2017-01-01
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,Ecuador,3QwBODjSEzelZyVjxPOHdq,2017-01-01
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,Ecuador,7DM4BPaS7uofFul3ywMe46,2017-01-01
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,Ecuador,6rQSrBHf7HlZjtcMZ4S4bO,2017-01-01


In [7]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19724913 entries, 0 to 34399
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Position      int64         
 1   Track Name    object        
 2   Artist        object        
 3   Streams       int64         
 4   URL           object        
 5   Date          datetime64[ns]
 6   Region        object        
 7   song_id       object        
 8   release_date  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 1.5+ GB


In [8]:
print(f'Num of songs: {songs_ts.song_id.nunique()}')
print(f'Max. total number of countries for a song: {songs_ts.total_countries.max()}')
print(f'Min. total number of countries for a song: {songs_ts.total_countries.min()}')
print(f'Max. current number of countries for a song: {songs_ts.current_countries.max()}')
print(f'Max. increment for a song: {songs_ts.inc.max()}')
print(f'Max. decrement for a song: {songs_ts.dec.max()}')

Num of songs: 4363
Max. total number of countries for a song: 53
Min. total number of countries for a song: 1
Max. current number of countries for a song: 52
Max. increment for a song: 10
Max. decrement for a song: 50


## Get songs' audio features

In [ ]:
song_features_lst = []
for song_id in tqdm(songs_ts['song_id'].unique()):
    try:
        release_date = ranking_df[ranking_df['song_id']==song_id]['release_date'].values[0]
        country = ranking_df[(ranking_df['song_id']==song_id)&(ranking_df['Date']==release_date)]['Region'].values[0]

        song_features = sp.audio_features(song_id)[0]
        song_features['release_date'] = release_date
        song_features['home_country']= country
        

        #print(song_features)
        song_features_lst.append(song_features)
    except:
        print("Error with song:", song_id)

In [ ]:
song_features_df = pd.DataFrame(song_features_lst)

In [ ]:
song_features_df.head()

In [ ]:
song_features_df.info()

In [ ]:
song_features_df.to_csv(os.path.join('data', 'songs_features_2017_2021.csv'))

## Get songs' genres

In [9]:
all_genres = sp.recommendation_genre_seeds()
all_genres= all_genres['genres']
all_genres += ['rap', 'pop rock', 'trap', 'ranchera', 'corrido', 'cumbia']
all_genres

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [10]:
gender_converter = {'r&b':'r-n-b', 'soundtrack':'soundtracks', 'ost':'soundtracks'}

In [11]:
song_genres_dict = {}
for song_id in tqdm(songs_ts['song_id'].unique()):
    #try:
    track_info= sp.track(song_id)
    track_genres = []
    for artist in track_info['artists']:
        artist_id=artist['id']
        artist_info= sp.artist(artist_id)
        artist_genres = ['hip-hop' if 'hip hop' in g else g for g in artist_info['genres']]
        artist_genres = [gender_converter.get(g,g) for g in artist_genres]
         
        track_genres += artist_genres #artist_info['genres']
        
        
    filtered_genres = [g for g in  track_genres if g in all_genres]
    
    if len(filtered_genres)==0:
        #print(track_genres)
        split_genres = [g.split(' ')[-1] for g in track_genres if len(g.split(' '))>1]
        split_genres += [g.split(' ')[0] for g in track_genres if len(g.split(' '))>1]
        split_genres = [gender_converter.get(g,g) for g in split_genres]
        
        filtered_genres = [g for g in split_genres if g in all_genres]

    if len(filtered_genres)==0:
        print(f'No genres from {track_genres}')
    song_genres_dict[song_id]= set(filtered_genres)
    sleep(0.01)
    #except:
    #    print("Error with song:", song_id)

  0%|          | 0/4363 [00:00<?, ?it/s]

No genres from ['musical advocacy']
No genres from []
No genres from ['adult standards', 'easy listening']
No genres from []
No genres from []
No genres from []
No genres from ['adult standards', 'easy listening', 'lounge']
No genres from ['hollywood', 'show tunes', 'hollywood']
No genres from ['adult standards', 'easy listening']
No genres from []
No genres from []
No genres from []
No genres from ['brooklyn drill']
No genres from ['brooklyn drill']
No genres from []
No genres from ['cabaret', 'movie tunes']
No genres from ['a cappella']
No genres from []
No genres from ['adult standards', 'easy listening']
No genres from ['adult standards', 'american folk revival']
No genres from ['adult standards', 'easy listening']
No genres from ['adult standards', 'easy listening']
No genres from []
No genres from ['alt z']
No genres from ['shanty']
No genres from ['new york drill']
No genres from ['cartoon', "children's music"]
No genres from ['basshall', 'francoton']
No genres from ['eurovision

No genres from []
No genres from ['weirdcore']
No genres from []
No genres from []
No genres from []
No genres from ['norteno', 'nuevo regional mexicano', 'regional mexican', 'sierreno', 'nuevo regional mexicano', 'regional mexican', 'sierreno']
No genres from ['corridos tumbados', 'nuevo regional mexicano', 'regional mexican', 'sierreno']
No genres from []
No genres from ['regional mexican']
No genres from []
No genres from []
No genres from ['chinese new year']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['chinese new year']
No genres from []
No genres from []
No genres from ['kleinkunst']
No genres from ['shatta', 'dutch moombahton']
No genres from []
No genres from ['scandipop']
No genres from ['fotbollslatar', 'russelater', 'russelater', 'melbourne bounce international']
No genres from []
No genres from ['fotbollslatar', 'russelater', 'russelater', 'melbourne bounce international']
No genres from ['fotbollslatar', 'russelater']
No genres 

In [12]:
song_genres_dict

{'4O4Z8VFczL8MxIOmqVWc1b': {'rock'},
 '7Il2yWQmsqKgSkLQvHTPUI': {'country', 'pop'},
 '6tHWl8ows5JOZq9Yfaqn3M': {'pop'},
 '3p0Y1C5jhaqyKW7d9oskXA': {'hip-hop', 'rap'},
 '2yQZwi1P8AkkxxFhQ8rMEK': {'country', 'pop'},
 '2ZxTjs4EdJl4Y9YByIiNoo': {'country'},
 '5lZsh9Qf7CbHI9Fcc7Zcsq': {'rap', 'trap'},
 '6hTQD2DUPajxh5lObhH9y1': {'edm', 'hip-hop', 'r-n-b', 'rap', 'trap'},
 '3KYOGt8IO6j8wptkG9THag': {'pop'},
 '11GGSyTB8cS9TgONHBOhEX': {'pop'},
 '43V1z6QToNNWNldV4FEG6r': {'edm', 'pop'},
 '1HehGmky06OZLM9Vz2PRZL': {'country'},
 '0CZ8lquoTX2Dkg7Ak2inwA': {'pop'},
 '4lTiuD1DwBKo4L7wqMkD7C': {'hip-hop', 'rap', 'trap'},
 '4LiMQ6G9n84rDUBs9wtwz6': {'country'},
 '7ms3VzLTfr7fXNOqA4Mkx9': {'hip-hop', 'rap', 'trap'},
 '7jpdFaZYgx93A67hLe8kwc': {'hip-hop', 'pop', 'rap', 'trap'},
 '5Qx9hngUBr30JFrNua7yp4': {'trap'},
 '7ozYT2gIHVbE9MBzFyzoIK': {'rock'},
 '79LAFAkR1ATb4iIQvZuCL0': {'country'},
 '1dNyRyz1QBN9kO1etUs1PI': {'rap'},
 '32iYwQ4OYiurnGnOfJ1aEh': {'rap'},
 '2R7Gn3cjnPaVO0Oj8nsTDP': {'hip-hop', 'po

In [13]:
song_genres_df = pd.DataFrame.from_dict(song_genres_dict, orient='index')
#song_genres_df.columns= 'track_genres'

In [14]:
song_genres_df.head()

,0,1,2,3,4,5
4O4Z8VFczL8MxIOmqVWc1b,rock,None,None,None,None,None
7Il2yWQmsqKgSkLQvHTPUI,pop,country,None,None,None,None
6tHWl8ows5JOZq9Yfaqn3M,pop,None,None,None,None,None
3p0Y1C5jhaqyKW7d9oskXA,rap,hip-hop,None,None,None,None
2yQZwi1P8AkkxxFhQ8rMEK,pop,country,None,None,None,None


In [15]:
song_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4363 entries, 4O4Z8VFczL8MxIOmqVWc1b to 3hpR0IWUJMvtlq3aAyKPqz
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3975 non-null   object
 1   1       1550 non-null   object
 2   2       597 non-null    object
 3   3       146 non-null    object
 4   4       46 non-null     object
 5   5       4 non-null      object
dtypes: object(6)
memory usage: 238.6+ KB


In [16]:
song_genres_df.to_csv(os.path.join('data', 'songs_genres_2017_2021.csv'))

## Get songs language

In [3]:
ranking_lite = ranking_df.drop_duplicates('song_id')
ranking_lite.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,Ecuador,3AEZUABDXNtecAOSC1qTfo,2017-01-01
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,Ecuador,6mICuAdrwEjh6Y6lroV2Kg,2017-01-01
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,Ecuador,3QwBODjSEzelZyVjxPOHdq,2017-01-01
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,Ecuador,7DM4BPaS7uofFul3ywMe46,2017-01-01
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,Ecuador,6rQSrBHf7HlZjtcMZ4S4bO,2017-01-01


In [4]:
ranking_lite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89402 entries, 0 to 249272
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Position      89402 non-null  int64         
 1   Track Name    89378 non-null  object        
 2   Artist        89379 non-null  object        
 3   Streams       89402 non-null  int64         
 4   URL           89402 non-null  object        
 5   Date          89402 non-null  datetime64[ns]
 6   Region        89402 non-null  object        
 7   song_id       89402 non-null  object        
 8   release_date  89402 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 6.8+ MB


In [7]:
language_short_name = {'aa':'Afar','ab':'Abkhazian','af':'Afrikaans','ak':'Akan','sq':'Albanian','am':'Amharic','ar':'Arabic',
'an':'Aragonese','hy':'Armenian','as':'Assamese','av':'Avaric','ae':'Avestan','ay':'Aymara','az':'Azerbaijani','ba':'Bashkir',
'bm':'Bambara','eu':'Basque','be':'Belarusian','bn':'Bengali','bh':'Bihari languages','bi':'Bislama','bo':'Tibetan','bs':'Bosnian',
'br':'Breton','bg':'Bulgarian','my':'Burmese','ca':'Catalan; Valencian','cs':'Czech','ch':'Chamorro','ce':'Chechen','zh':'Chinese',
'cu':'Church Slavic; Old Slavonic; Church Slavonic; Old Bulgarian; Old Church Slavonic','cv':'Chuvash','kw':'Cornish','co':'Corsican',
'cr':'Cree','cy':'Welsh','cs':'Czech','da':'Danish','de':'German','dv':'Divehi; Dhivehi; Maldivian','nl':'Dutch; Flemish','dz':'Dzongkha',
'el':'Greek-Modern (1453-)','en':'English','eo':'Esperanto','et':'Estonian','eu':'Basque','ee':'Ewe','fo':'Faroese','fa':'Persian',
'fj':'Fijian','fi':'Finnish','fr':'French','fy':'Western Frisian','ff':'Fulah','Ga':'Georgian','gd':'Gaelic; Scottish Gaelic','ga':'Irish',
'gl':'Galician','gv':'Manx','el':'Greek-Modern (1453-)','gn':'Guarani','gu':'Gujarati','ht':'Haitian; Haitian Creole','ha':'Hausa',
'he':'Hebrew','hz':'Herero','hi':'Hindi','ho':'Hiri Motu','hr':'Croatian','hu':'Hungarian','hy':'Armenian','ig':'Igbo','is':'Icelandic',
'io':'Ido','ii':'Sichuan Yi; Nuosu','iu':'Inuktitut','ie':'Interlingue; Occidental',
'ia':'Interlingua (International Auxiliary Language Association)','id':'Indonesian','ik':'Inupiaq','is':'Icelandic','it':'Italian',
'jv':'Javanese','ja':'Japanese','kl':'Kalaallisut; Greenlandic','kn':'Kannada','ks':'Kashmiri','ka':'Georgian','kr':'Kanuri','kk':'Kazakh',
'km':'Central Khmer','ki':'Kikuyu; Gikuyu','rw':'Kinyarwanda','ky':'Kirghiz; Kyrgyz','kv':'Komi','kg':'Kongo','ko':'Korean',
'kj':'Kuanyama; Kwanyama','ku':'Kurdish','lo':'Lao','la':'Latin','lv':'Latvian','li':'Limburgan; Limburger; Limburgish','ln':'Lingala',
'lt':'Lithuanian','lb':'Luxembourgish; Letzeburgesch','lu':'Luba-Katanga','lg':'Ganda','mk':'Macedonian','mh':'Marshallese',
'ml':'Malayalam','mi':'Maori','mr':'Marathi','ms':'Malay','Mi':'Micmac','mk':'Macedonian','mg':'Malagasy','mt':'Maltese',
'mn':'Mongolian','mi':'Maori','ms':'Malay','my':'Burmese','na':'Nauru','nv':'Navajo; Navaho','nr':'Ndebele-South; South Ndebele',
'nd':'Ndebele-North; North Ndebele','ng':'Ndonga','ne':'Nepali','nl':'Dutch; Flemish','nn':'Norwegian Nynorsk; Nynorsk:Norwegian',
'nb':'Bokmål-Norwegian; Norwegian Bokmål','no':'Norwegian','oc':'Occitan (post 1500)','oj':'Ojibwa','or':'Oriya','om':'Oromo',
'os':'Ossetian; Ossetic','pa':'Panjabi; Punjabi','fa':'Persian','pi':'Pali','pl':'Polish','pt':'Portuguese','ps':'Pushto; Pashto',
'qu':'Quechua','rm':'Romansh','ro':'Romanian; Moldavian; Moldovan','rn':'Rundi','ru':'Russian','sg':'Sango','sa':'Sanskrit',
'si':'Sinhala; Sinhalese','sk':'Slovak','sk':'Slovak','sl':'Slovenian','se':'Northern Sami','sm':'Samoan','sn':'Shona','sd':'Sindhi',
'so':'Somali','st':'Sotho-Southern','es':'Spanish; Castilian','sq':'Albanian','sc':'Sardinian','sr':'Serbian','ss':'Swati',
'su':'Sundanese','sw':'Swahili','sv':'Swedish','ty':'Tahitian','ta':'Tamil','tt':'Tatar','te':'Telugu','tg':'Tajik','tl':'Tagalog',
'th':'Thai','bo':'Tibetan','ti':'Tigrinya','to':'Tonga (Tonga Islands)','tn':'Tswana','ts':'Tsonga','tk':'Turkmen','tr':'Turkish',
'tw':'Twi','ug':'Uighur; Uyghur','uk':'Ukrainian','ur':'Urdu','uz':'Uzbek','ve':'Venda','vi':'Vietnamese','vo':'Volapük','cy':'Welsh',
'wa':'Walloon','wo':'Wolof','xh':'Xhosa','yi':'Yiddish','yo':'Yoruba','za':'Zhuang; Chuang','zh':'Chinese','zu':'Zulu'}

In [14]:
"""
songs_lang = []
for i,r in ranking_lite.iterrows():
    song_title = r['Track Name']
    song_id = r['song_id']
    #if len(song_title)>=3:
        #b = TextBlob(song_title)
        #song_lang = b.detect_language()
    #song_lang= langid.classify(song_title)
    #song_lang= cld3.get_language(song_title)
    #song_lang= detect(song_title)
    
    nlp = spacy.load('en')  # 1
    nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) #2
    doc = nlp(song_title) #3
    song_lang = doc._.language #4
    #print(song_lang)
    song_lang_verbose = language_short_name[song_lang['language']]

    print(f'{song_id}, {song_title}, {song_lang_verbose}')
    songs_lang.append((song_id, song_lang))
    #else:
    #print(f'Title too sort for {song_id}: {song_title}')
    #sleep(0.05)
"""

3AEZUABDXNtecAOSC1qTfo, Reggaetón Lento (Bailemos), Spanish; Castilian
6mICuAdrwEjh6Y6lroV2Kg, Chantaje, Croatian
3QwBODjSEzelZyVjxPOHdq, Otra Vez (feat. J Balvin), Romanian; Moldavian; Moldovan
7DM4BPaS7uofFul3ywMe46, Vente Pa' Ca, Portuguese
6rQSrBHf7HlZjtcMZ4S4bO, Safari, Somali
0sXvAOmXgjR2QUqLK1MltU, La Bicicleta, Catalan; Valencian
6stYbAJgTszHAHZMPxWWCY, Ay Mi Dios, Tagalog
5mey7CLLuFToM2P68Qu1gF, Andas En Mi Cabeza, Spanish; Castilian
5J1c3M4EldCfNxXwrwt8mT, Traicionera, Italian
58IL315gMSTD37DOZPJ2hf, Shaky Shaky, Somali
3dQDid3IUNhZy1OehIfYfE, Vacaciones, Spanish; Castilian
20ZAJdsKB5IGbGj4ilRt2o, Dile Que Tu Me Quieres, French
4pdPtRcBmOSQDlJ3Fk945m, Let Me Love You, French
6YZdkObH88npeKrrkb8Ggf, DUELE EL CORAZON, Catalan; Valencian
1lxswgIpzV6HhENRvkflES, Chillax, English
6DUdDIRgLqCGq1DwkNWQTN, Borro Cassette, Italian
1xznGGDReH1oQq0xzbwXa3, One Dance, French
7BKLCZ1jbUBVqRi2FVlTVw, Closer, Spanish; Castilian
5aAx2yezTd8zXrkmtKl66Z, Starboy, Swedish
1pWYnQIlqxTh5bxuPmTG4E

1WOJTQfNUEkLlsinPP95VM, Embriágame (feat. Don Omar) - Remix, Romanian; Moldavian; Moldovan
11KJSRSgaDxqydKYiD2Jew, Too Good, Somali
3pzjHKrQSvXGHQ98dx18HI, What Do You Mean?, English
47TyfLrGhYHHyyQWk64d0M, Flor Pálida, Portuguese
6ZvmnulA6jJY3lvveuCz77, Tan Fácil - Urban Remix, Indonesian
4TZy1wLyHec06pwgFYDh1a, Mercy, Welsh
2xNZeeqmwMPVKnD7FSQfgt, Sin Contrato - Remix, Italian
1D3ODoXHBLpdxolZRHWV1j, By Your Side, English
06sGPlylSN84nEQK23fZhQ, Lloraras, Catalan; Valencian
5uCax9HTNlzGybIStD3vDh, Say You Won't Let Go, English
0CrQ7dNvGPXmex621Waet8, Te Encontre, Spanish; Castilian
0XXLwO7YHHTAK3OJkWQ37b, Sorry - Latino Remix, English
0JiVRyTJcJnmlwCZ854K4p, Alone, Italian
32OlwWuMpZ6b0aN2RZOeMS, Uptown Funk, English
1nZziTuPRwCfJe1XvVC760, Yo No Sé Mañana (salsa), Tagalog
7kxYYjOYRO8HamP4RKBhJ0, Si No Te Quiere, German
0VjbN2rfs40mHQ5mZN9UCE, Me Quema, French
343YBumqHu19cGoGARUTsd, Fake Love, Slovenian
5HJyn7mTQq7OtmpldocdZC, Oiga, Mira, Vea, Portuguese
54MeG5FCVXStJTyNvRu9zN, Love

4PdJSsESm34djLfBde9Pr2, Would I Lie To You - Radio Edit, English
77a9893yxuz36cBUmBzA8E, Al Filo de Tu Amor, Portuguese
58EQDn2K9Gyj68YIckOLrN, El Chisme (feat. J Alvarez & Kevin Roldan) - Remix, English
5XXxxfDWGvI6OaXPEniklV, Quédate, French
3NaEzk0OFnezhFFGqZNyYz, Destino o Casualidad, Portuguese
2LsbizbOeNa4x6qsi2jAMb, La flaca, Spanish; Castilian
3ebXMykcMXOcLeJ9xZ17XH, Scared to Be Lonely, English
6oV8HazS5rJB3jU3oPhYep, Lo Niego Todo, Polish
3H0Vda4LvcNlwRQpNd8MHJ, Hermosa Ingrata, Tagalog
6p3yxt0uJJshAS3CT0znQg, Tú Sí Sabes Quererme, Catalan; Valencian
0fg8CqpjdojMyXLNzM2PaJ, Run Up (feat. PARTYNEXTDOOR & Nicki Minaj), Polish
1sXUWdKx03aP9Gmzft58rt, Issues, French
3E2Zh20GDCR9B1EYjfXWyv, Weak, Swahili
4Km5HrUvYTaSUfiSGPJeQR, Bad and Boujee (feat. Lil Uzi Vert), German
5qqabIl2vWzo9ApSC317sa, Wonderwall - Remastered, German
2tergQtVsHUBYlL76Kx93c, Olvídame y Pega la Vuelta, Spanish; Castilian
7yjTvUlvx7S1pneUODlGBg, La Casa No Es Igual, Spanish; Castilian
3vsNteZn6E5qBSSBO78W7b,

5g7sDjBhZ4I3gcFIpkrLuI, Locked Out Of Heaven, English
0Szp49tpFasIjX04Mcsydp, Tu Foto, Croatian
2Gl0FzuLxflY6nPifJp5Dr, First Time, English
61WbtB6ujkpNAsAf5LjF4b, Bon appétit, French
0tKcYR2II1VCQWT79i5NrW, Thunder, German
3DXncPQOG4VBw3QHh3S817, I'm the One, English
2bBMODHVTXGwE2aZYfbgCt, Ascension (feat. Vince Staples), English


KeyError: 'UNKNOWN'

In [ ]:
print("That's all folks!")